In [19]:
import numpy as np

In [20]:
def P2R(radii, angles) -> complex:
    return np.round(radii * np.exp(1j*angles), 3)

def R2P(x: complex, string: bool = False, sym: str = "", base: float = 1) -> list[float]|str:
    if not string:
        return [round(abs(x), 3), round(np.angle(x), 3)]
    else:
        return (
            f"{round(abs(x) * base, 3):>6} {sym} ∠ {round(np.angle(x), 3):>6}"
            u"\N{DEGREE SIGN}"
        )

In [21]:
s_base: float = 100 * 10**6         # 100 MVA base
v_base: float = 400 * 10**3         # 400 kV base
z_base: float = v_base**2 / s_base  # Z = V^2 / S
i_base: float = s_base / v_base     # I = S / V

In [22]:
line_1_2: complex = (0.00 + (0.64j * 100)) / z_base     # impedance from bus 1 to bus 2
line_1_3: complex = (0.00 + (0.64j * 62.5)) / z_base    # impedance from bus 1 to bus 3
line_2_3: complex = (0.00 + (0.64j * 50)) / z_base      # impedance from bus 2 to bus 3

# Construct impedance matrix
imp_matrix: list[complex] = [
    line_1_2,
    line_1_3,
    line_2_3,
]

imp_matrix

[0.04j, 0.025j, 0.02j]

In [53]:
# Convert impedance to admittances
add_line_1_2: complex = 1 / line_1_2  
add_line_1_3: complex = 1 / line_1_3
add_line_2_3: complex = 1 / line_2_3

# Create self admittances
Y_11: complex = add_line_1_2 + add_line_1_3
Y_22: complex = add_line_1_2 + add_line_2_3
Y_33: complex = add_line_1_3 + add_line_2_3

# Create mutual admittances
Y_12: complex = -add_line_1_2
Y_13: complex = -add_line_1_3

Y_21: complex = -add_line_1_2
Y_23: complex = -add_line_2_3

Y_31: complex = -add_line_1_3
Y_32: complex = -add_line_2_3

# Construct admittance matrix
admittance_matrix: np.ndarray = np.array(
    [
        [Y_11, Y_12, Y_13],
        [Y_21, Y_22, Y_23],
        [Y_31, Y_32, Y_33]
    ]
)

print("Admittance Matrix")
for i in admittance_matrix:
    print(f"{"|":^1} {i[0].imag:>6.2f}j {i[1].imag:>6.2f}j {i[2].imag:>6.2f}j {"|":^1}")

| -65.00j  25.00j  40.00j |
|  25.00j -75.00j  50.00j |
|  40.00j  50.00j -90.00j |


In [55]:
emf_1: complex = P2R(1.000, 0.000)      # EMF at bus 1
emf_2: complex = P2R(0.986, -1.799)     # EMF at bus 2
emf_3: complex = P2R(1.000, -0.345)     # EMF at bus 3

# Create EMF matrix
emf_matrix = np.array(
    [
        [emf_1],
        [emf_2],
        [emf_3]
    ]
)

# Print out EMF values in rectangular form
print(f"{"|":^1} {"Bus":^6} {"|":^1} {"EMF":^19} {"|":^1}")
print(f"{"|":^1} {"------"} {"|":^1} {"-------------------"} {"|":^1}")
for i in range(len(emf_matrix)):
    neg_sign_real = False if emf_matrix[i][0].real >= 0 else True
    neg_sign_imag = False if emf_matrix[i][0].imag >= 0 else True
    print(f"{"|":1} {i + 1:^6} {"|":^1} "
        f"{"-" if neg_sign_real == True else "+":^3}"
        f"{abs(emf_matrix[i][0].real):^4.3f}"
        f"{"-" if neg_sign_imag == True else "+":^3}"
        f"{abs(emf_matrix[i][0].imag):^4.3f}"
        f"{"j V":^3}"
        f" {"|":^1}"
    )

|  Bus   |         EMF         |
| ------ | ------------------- |
|   1    |  + 1.000 + 0.000j V |
|   2    |  - 0.223 - 0.960j V |
|   3    |  + 0.941 - 0.338j V |


In [49]:
10# Calculate branch currents
branch_currents = {
    "I_12": imp_matrix[0] * (emf_matrix[0][0] - emf_matrix[1][0]),
    "I_13": imp_matrix[1] * (emf_matrix[0][0] - emf_matrix[2][0]),
    "I_21": imp_matrix[0] * (emf_matrix[1][0] - emf_matrix[0][0]),
    "I_23": imp_matrix[2] * (emf_matrix[1][0] - emf_matrix[2][0]),
    "I_31": imp_matrix[1] * (emf_matrix[2][0] - emf_matrix[0][0]),
    "I_32": imp_matrix[2] * (emf_matrix[2][0] - emf_matrix[1][0]),
}

# Print branch currents in PU
print("Branch Currents in PU")
for key, value in branch_currents.items():
    print(f"{key}: {R2P(value, True, "")}")
print("\n")

# Print branch currents in Amperes
print("Branch Currents in Amps")
for key, value in branch_currents.items():
    print(f"{key}: {R2P(value, True, "A", i_base)}")

Branch Currents in PU
I_12:  0.062 A ∠  2.236°
I_13:  0.009 A ∠  2.969°
I_21:  0.062 A ∠ -0.905°
I_23:  0.026 A ∠  -1.08°
I_31:  0.009 A ∠ -0.173°
I_32:  0.026 A ∠  2.062°


Branch Currents in Amps
I_12: 15.548 A ∠  2.236°
I_13:  2.144 A ∠  2.969°
I_21: 15.548 A ∠ -0.905°
I_23:  6.599 A ∠  -1.08°
I_31:  2.144 A ∠ -0.173°
I_32:  6.599 A ∠  2.062°


In [59]:
# Branch power flows
apparent_power = {
    "S_12": emf_matrix[0][0] * np.conj(branch_currents["I_12"]),
    "S_13": emf_matrix[0][0] * np.conj(branch_currents["I_13"]),
    "S_21": emf_matrix[1][0] * np.conj(branch_currents["I_21"]),
    "S_23": emf_matrix[1][0] * np.conj(branch_currents["I_23"]),
    "S_31": emf_matrix[2][0] * np.conj(branch_currents["I_31"]),
    "S_32": emf_matrix[2][0] * np.conj(branch_currents["I_32"]),
}

print("Branch Power Flows in MVA")
for key, value in apparent_power.items():
    print(f"{key}: {R2P(value, True, "MVA", 100)}")
print("\n")

print("Branch Power Flows in MW and MVAr")
for key, value in apparent_power.items():
    print(f"{key}: {round(value.real * 100, 3):>6} MW & {round(value.imag * 100, 3):>6} MVAr")

Branch Power Flows in MVA
S_12:  6.219 MVA ∠ -2.236°
S_13:  0.858 MVA ∠ -2.969°
S_21:  6.129 MVA ∠ -0.894°
S_23:  2.601 MVA ∠ -0.719°
S_31:  0.858 MVA ∠ -0.172°
S_32:  2.639 MVA ∠ -2.406°


Branch Power Flows in MW and MVAr
S_12:  -3.84 MW & -4.892 MVAr
S_13: -0.845 MW & -0.148 MVAr
S_21:   3.84 MW & -4.777 MVAr
S_23:  1.957 MW & -1.713 MVAr
S_31:  0.845 MW & -0.147 MVAr
S_32: -1.957 MW &  -1.77 MVAr


In [56]:
bus_1 = P2R(1.000, 0.000)
bus_2 = P2R(0.986, -1.799)
bus_3 = P2R(1.000, -0.345)

vol_1_2 = R2P(bus_1 - bus_2, True, "kV", v_base/1000)
vol_1_3 = R2P(bus_1 - bus_3, True, "kV", v_base/1000)
vol_2_3 = R2P(bus_2 - bus_3, True, "kV", v_base/1000)

print(f"voltage 1 -> 2: {vol_1_2}")
print(f"voltage 1 -> 3: {vol_1_3}")
print(f"voltage 1 -> 2: {vol_2_3}")


voltage 1 -> 2: 621.91 kV ∠  0.665°
voltage 1 -> 3: 137.244 kV ∠  1.398°
voltage 1 -> 2: 527.906 kV ∠ -2.651°
